## Simple JSON Q&A Approach (LangChain)

When we have a question / answer dataset with relatively short answers, we don't need to use a splitter



In [2]:
import json
# Load JSON data
with open("./data/home0001qa.json", "r") as file:
    qa_data = json.load(file)

In [ ]:
from langchain.schema import Document
# Prepare documents for LangChain
documents = [
    Document(page_content=item["answer"], metadata={"question": item["question"]})
    for item in qa_data
]

documents[:5]

In [ ]:
# turn into function
def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content=item["answer"],
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

print(prepare_qa_documents("./data/home0001qa.json")[:5])

In [23]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def setup_rag(documents):
    
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents, embeddings)
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following similar Q&A pairs to help answer the question. 
        If the context is relevant, use it to answer. If not, say you don't have enough information.
        
        Context Q&A pairs:
        {context}
        """),
        ("human", "{question}")
    ])
    
    chain = (
        {"context": vectorstore.as_retriever(search_type="similarity", k=3), "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [ ]:
docs = prepare_qa_documents('./data/home0001qa.json')
chain = setup_rag(docs)
print(chain.invoke("Do i own my 0001 home outright?"))

## Basic RAG pipeline w/ Memory (LangGraph)
https://python.langchain.com/docs/tutorials/qa_chat_history/

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()
documents = prepare_qa_documents("./data/home0001qa.json") # from above
vectorstore = Chroma.from_documents(documents, embeddings)
document_retriever = vectorstore.as_retriever()

# System prompt that explains how to handle questions with chat history
history_context_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
# Create prompt template for contextualizing questions
history_contextualization_template = ChatPromptTemplate.from_messages(
    [
        ("system", history_context_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, 
    document_retriever, 
    history_contextualization_template
)

### Answer question ###
answer_generation_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
answer_generation_template = ChatPromptTemplate.from_messages(
    [
        ("system", answer_generation_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create chain for generating answers from documents
question_answer_chain = create_stuff_documents_chain(llm, answer_generation_template)

# Create the complete RAG chain with memory
rag_pipeline_with_memory = create_retrieval_chain(
    history_aware_retriever, 
    question_answer_chain
)

User Question  
↓  
Add Chat History Context  
↓  
Find Relevant Documents  
↓  
Generate Answer  
↓  
Return Response

In [12]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing import Sequence
from typing_extensions import Annotated, TypedDict

### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_pipeline_with_memory.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": "hi my name is Flippy, what is home001?"},
    config=config,
)
print(result["answer"])

In [ ]:
result = app.invoke(
    {"input": "what's my name?"},
    config=config,
)
print(result["answer"])

Idea: Update Chat History w/ Operator corrected messages?

In [ ]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

### All in one Agentic Workflow

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()
documents = prepare_qa_documents("./data/home0001qa.json") # from above
memory = MemorySaver()
vectorstore = Chroma.from_documents(documents, embeddings)
document_retriever = vectorstore.as_retriever()



## Manual LOCAL Document Grader (LangChain)
https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag_local/

In [ ]:
# Doc grader instructions
doc_grader_instructions = """
You are a grader assessing relevance of a retrieved document to a user question.
If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
"""

# Grader prompt
doc_grader_prompt = """
Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

This carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question.
"""

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

llm_model = "llama3.2:3b-instruct-fp16"

llm_json_mode = ChatOllama(model=llm_model, temperature=0, format='json')

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever(k=3)

# Test
question = "Does furniture come included?"
docs = retriever.invoke(question)
doc_txt = docs[0].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)

print(doc_txt)
print(json.loads(result.content))

In [ ]:
def grade_documents(documents, question):
  
    # Score each doc
    filtered_docs = []

    for d in documents:
        doc_grader_prompt_formatted = doc_grader_prompt.format(
            document=d.page_content, question=question
        )
        result = llm_json_mode.invoke(
            [SystemMessage(content=doc_grader_instructions)]
            + [HumanMessage(content=doc_grader_prompt_formatted)]
        )
        grade = json.loads(result.content)["binary_score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            continue

    return {"documents": filtered_docs}

In [ ]:
question = "Do i own my 0001 home outright?"
retrieved_docs = retriever.invoke(question)
filtered_docs = grade_documents(retrieved_docs, question)

In [ ]:
# TO DO: 
# ANSWER GRADER 

## Links (LangChain)

pip install --upgrade --quiet lark chromadb  
restart kernel  

In [18]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

docs = [
    Document(
        page_content="A 1 Bedroom apartment in New York",
        metadata={"link": "https://www.home0001.com/property-type/1-bedroom", "area": "Lower East Side"},
    ),
    Document(
        page_content="A 1 Bedroom apartment in Paris",
        metadata={"link": "https://www.home0001.com/property-type/1-bdrm-berlin", "area": "Schoeneberg"},
    ),
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

In [19]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="link",
        description="The link to the property",
        type="string",
    ),
    AttributeInfo(
        name="area",
        description="The area the property is in",
        type="integer",
    )
]
document_content_description = "Brief overview of a property."
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 1}
)

In [ ]:
retriever.invoke("I want to buy a property in Schoeneberg", k=1) # k not working?



In [24]:
docs = [
    Document(
        page_content="A 1 Bedroom apartment in New York. The link to the Dossier is: https://www.home0001.com/property-type/1-bedroom",
        metadata={"link": "https://www.home0001.com/property-type/1-bedroom", "area": "Lower East Side"},
    ),
    Document(
        page_content="A 1 Bedroom apartment in Berlin. The link to the Dossier is: https://www.home0001.com/property-type/1-bdrm-berlin",
        metadata={"link": "https://www.home0001.com/property-type/1-bdrm-berlin", "area": "Schoeneberg"},
    ),
]

def setup_rag(documents):
    
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents, embeddings)
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following information to answer your question. If a user asks about a specific neighborhood or property, feel free to send the coresponding link.
        {context}
        """),
        ("human", "{question}")
    ])
    
    chain = (
        {"context": vectorstore.as_retriever(search_type="similarity", k=3), "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [ ]:
chain = setup_rag(docs)
print(chain.invoke("Do u have any places in berlin?"))

## Markdown Splitting
https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/  


In [32]:
def read_markdown_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "City"),
    ("##", "Neighborhood"),
    ("###", "Property Type"),
]

markdown_document = read_markdown_file("./data/properties.md")
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_document)
print(md_header_splits[:5])

Note: The structure with metadata allows for SelfQueryRetriever

Possible to constrain chunk size:  
Within each markdown group we can then apply any text splitter we want, such as RecursiveCharacterTextSplitter, which allows for further control of the chunk size.

In [39]:
def setup_rag(documents):
    
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents, embeddings)
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following information to answer your question. If a user asks about a specific neighborhood or property, feel free to send the corresponding link.
        {context}
        """),
        ("human", "{question}")
    ])
    
    chain = (
        {"context": vectorstore.as_retriever(search_type="similarity", k=3), "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [40]:
chain = setup_rag(md_header_splits)

In [ ]:
print(chain.invoke("Do u have a 1 bedroom in LES?"))

## Agentic RAG

In [48]:
from langchain_community.vectorstores import FAISS

def local_vector_db(documents, embeddings):

    vectorstore = FAISS.from_documents(
        documents, 
        embeddings
    )
    vectorstore.save_local("./FAISS", "test-db")

    return vectorstore

In [49]:
from langchain.tools.retriever import create_retriever_tool

documents = md_header_splits
embeddings = OpenAIEmbeddings()
vectorstore = local_vector_db(documents, embeddings)
retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "h0001_retriever",
    "Search and return information about HOME0001 properties.",
)
tools = [retriever_tool]

In [ ]:
retriever_tool.invoke("Do u have a 1 bedroom in Lower East Side?")


## Combined Data w/ Self Querying Retriever

Retriever that uses a vector store and an LLM to generate the vector store queries.



In [1]:
import json
from langchain_core.documents import Document


def prepare_json_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content=item["answer"],
            metadata={"question": item["question"], "source": "FAQ"}
        )
        for item in qa_data
    ]
    
    return documents

In [3]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

def prepare_markdown_documents(file_path):

    headers_to_split_on = [
        ("#", "City"),
        ("##", "Neighborhood"),
        ("###", "Property Type"),
    ]

    with open(file_path, "r", encoding="utf-8") as file:
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
        documents = markdown_splitter.split_text(file.read())

    # Add "source": "Properties" to the metadata of each document
    for document in documents:
        document.metadata["source"] = "Properties"

    return documents

In [ ]:
json_documents = prepare_json_documents("./data/home0001qa.json")
markdown_documents = prepare_markdown_documents("./data/properties.md")
print(markdown_documents[:1])

# Combine both lists into one
combined_documents = json_documents + markdown_documents

In [30]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    combined_documents,
    embeddings,
    # persist_directory="./chroma_langchain_db"
  )

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def setup_rag(vectorstore):
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following information to answer your question. If a user asks about a specific neighborhood or property, feel free to send the corresponding link.
        {context}
        """),
        ("human", "{question}")
    ])
    
    chain = (
        {"context": vectorstore.as_retriever(search_type="similarity", k=3), "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [32]:
chain = setup_rag(vectorstore)

In [ ]:
print(chain.invoke("do you have a dossier avilable for a studio apartment in the lower east side?"))

In [42]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo


llm = ChatOpenAI(
    model="gpt-4o"
)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

rag_prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following information to answer your question. If a user asks about a specific neighborhood or property, feel free to send the corresponding link.
        {context}
        """),
        ("human", "{question}")
    ])

document_content_description = "Information about the housing collective HOME0001 and their available properties."

metadata_field_info = [
        AttributeInfo(
            name="source",
            description="Type of information source. Either FAQ or Properties. Use Properties for info about available properties and Neighborhoods. Use FAQ for general questions",
            type="string",
        ),
        AttributeInfo(
            name="City",
            description="The City a property is in.",
            type="string",
        ),
        AttributeInfo(
            name="Property Type",
            description="The kind of property (e.g. One Bedroom, Studio, etc.).",
            type="string",
        )
    ]

retriever = SelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
        search_kwargs={"k": 2}
    )

# Define application steps
def retrieve(state: State):
    retrieved_docs = retriever.invoke(state["question"])
    print(retrieved_docs)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = rag_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

def setup_graph():

    # Compile application and test
    graph_builder = StateGraph(State).add_sequence([retrieve, generate])
    graph_builder.add_edge(START, "retrieve")
    graph = graph_builder.compile()

    return graph


In [ ]:
graph = setup_graph()
response = graph.invoke({"question": "What is HOME0001?"})
print(response["answer"])

In [ ]:
response = graph.invoke({"question": "do you have a dossier avilable for a studio apartment in the lower east side?"})
print(response["answer"])


## Add Citations

In [45]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

rag_prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following information to answer your question. If a user asks about a specific neighborhood or property, feel free to send the corresponding link.
        {context}
        """),
        ("human", "{question}")
    ])

retriever =  vectorstore.as_retriever(search_type="similarity", k=3)
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = rag_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
result = graph.invoke({"question": "do u have a 1 bedroom in lower east side available?"})

sources = [doc.metadata["source"] for doc in result["context"]]
print(f"Sources: {sources}\n\n")
print(f'Answer: {result["answer"]}')